### **przyjazny bot**
#### post lyrics from top 50 songs in Poland on Twitter

Import necessary libraries

In [24]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import tweepy
import random
import os
import pandas as pd
from dotenv import load_dotenv

##### Spotify API Authorization

In [25]:
load_dotenv()

auth_manager = SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"), client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"))
sp = spotipy.Spotify(auth_manager=auth_manager)

Download track ids from specific playlist

In [26]:
def download_tracks_ids(username, playlist_id):
    tracks_ids = []
    playlist = sp.user_playlist(username, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        tracks_ids.append(track['id'])
    return tracks_ids

In [27]:
tracks_ids = download_tracks_ids(os.getenv("USERNAME"), os.getenv("PLAYLIST_ID"))
print(tracks_ids)

['3Nqxx5fdl3FQlrk3zarRmA', '09m29we7x9tgxyo6f3OQ87', '384lHduuDsrR4HPUwR3fG4', '4JmaOG7bfj1uNv8jEUg0Ce', '3f9Ugg030LDe0p6ospTORC', '3p6uAP2ajkobkIW8IkEpgr', '3nqQXoyQOWXiESFLlDF1hG', '4uUG5RXrOk84mYEfFvj3cK', '73vIOb4Q7YN6HeJTbscRx5', '2yxPqxHbj1L7vrCpHLtAcG', '08sMDBcjUBXWV99gZI092C', '5GdFPcbGDQqtbVO2jl6Ez5', '4nv5o5Xo4ySBaIVnXr75Xs', '1yiGGRGdHEc1F3jF5jEVmH', '561jH07mF1jHuk7KlaeF0s', '11BKm0j4eYoCPPpCONAVwA', '4N6rgD6EqLbWcl5ld3NqTP', '0zmxM4MXfisJRTQcPa1wbv', '7jtQIBanIiJOMS6RyCx6jZ', '4lPAZiTJbaQ4irMzcq0GOU', '11JXHNIH2xO5INcJQOWCjY', '53OQFPwyflA0jMyUjUSawr', '0JXXNGljqupsJaZsgSbMZV', '2KTfSSKBBfMLXwDqrtoOE5', '4h9wh7iOZ0GGn8QVp4RAOB', '5YvL3ytkusKcM5ozxD9fKN', '5NbxvSCL1xLqqWLqG6LjwE', '5XeFesFbtLpXzIVDNQP22n', '1cAAn5PKy40T1p15fpTPW8', '1Qrg8KqiBpW07V7PNxwwwL', '14154tJOTjeyeOyhnqP4QM', '2QjOHCTQ1Jl3zawyYOpxh6', '1VoUHMVxu1qG2Z9josOhWs', '2yEWVnp65tTXSrRW03IqXs', '1N8TTK1Uoy7UvQNUazfUt5', '3rb0tMq42WfggucPm0HHkA', '4Dvkj6JhhA12EX05fT7y2e', '5fjcRpwg94xQ2hrSqw6x06', '5fxFhJyveL

Draw of a track id

In [28]:
track_id = random.choice(tracks_ids)
print(track_id)

5fjcRpwg94xQ2hrSqw6x06


Collect data (title and main artist) needed to search lyrics via the Genius API

In [29]:
def artist_genius(track_id):
    track_info = sp.track(track_id)
    artist_genius = track_info['album']['artists'][0]['name']
    return artist_genius

In [30]:
def title(track_id):
    song_info = sp.track(track_id)
    title = song_info['name']
    return title

Collect information about all artists to include in a tweet

In [31]:
def artists(track_id):
    track_info = sp.track(track_id)
    artists_list = []
    for i in range(0, len(track_info['album']['artists'])):
        artists_list.append(track_info['album']['artists'][i]['name']) 
    str_artists = ", "
    return(str_artists.join(artists_list))

In [32]:
print(artist_genius(track_id), title(track_id), artists(track_id))

IGNACY Czekam Na Znak IGNACY


##### Genius API

In [33]:
genius = lyricsgenius.Genius(os.getenv("GENIUS_CLIENT_ID"))
genius.remove_section_headers = True

Download track lyrics from Genius

In [34]:
def raw_lyrics(track_id):
    lyrics = genius.search_song(title(track_id), artist_genius(track_id)).lyrics
    return lyrics

Processing the text to include it in the tweet

In [35]:
def tweet_content(lyrics, track_id):
    lyrics = lyrics.split('\n')
    for index in range(len(lyrics)):
        if lyrics[index] == "" or "[" in lyrics[index]:
            lyrics[index] = "XXX"
    lyrics = [i for i in lyrics if i != "XXX"]

    random_num = random.randrange(0, len(lyrics)-1)
    tweet = lyrics[random_num] + "\n" + lyrics[random_num+1] + "\n" + "\n" + title(track_id) + " by " + artists(track_id)
    tweet = tweet.replace("\\", "")
    return tweet

In [36]:
lyrics = raw_lyrics(track_id)
tweet_content(lyrics, track_id)

Searching for "Czekam Na Znak" by IGNACY...
Done.


'Bardzo chciałbym to zrozumieć\nNasze myśli dzieli wysoki mur\n\nCzekam Na Znak by IGNACY'

##### Twitter API

In [37]:
api = tweepy.Client(bearer_token=os.getenv("BEARER_TOKEN"), consumer_key= os.getenv("CONSUMER_KEY"), consumer_secret=os.getenv("CONSUMER_SECRET"), access_token=os.getenv("ACCESS_TOKEN"), access_token_secret=os.getenv("ACCESS_TOKEN_SECRET") )

Tweet lyrics of one of TOP 50 tracks in Poland

In [38]:
result = None
while result is None:
    try:
      lyrics = raw_lyrics(track_id)
      result = api.create_tweet(text=tweet_content(lyrics, track_id))
    except:
         pass

Searching for "Czekam Na Znak" by IGNACY...
Done.


#### Retweet quote tweets and reply on mention

Draw of a track id

In [39]:
track_id = random.choice(tracks_ids)
print(track_id)

4nng35mDbhJjPyGwXFNk76


In [40]:
def get_tweet(id):
    tweet = api.get_tweet(id, expansions=['author_id'], user_fields=['username'])
    return tweet

In [41]:
def quote_tweets(query, max_results):
    tweets = api.search_recent_tweets(query=query, max_results=max_results)
    results = []
    if not tweets.data is None and len(tweets.data) > 0:
        for tweet in tweets.data:
            twt = get_tweet(tweet['id'])
            obj = {}
            obj['id'] = tweet.id
            obj['username'] = twt.includes['users'][0].username
            obj['text'] = tweet.text
            results.append(obj)
    return results

Check if whether a tweet (id) has already received a reply

In [42]:
def check(id):
    with open('last_tweet_ids.txt') as f:
        datafile = f.readlines()
    for line in datafile:
        if id in line:
            return True
    return False

Add tweet (id) to database = text file


In [43]:
def append_new_id(text_to_append):
    with open('last_tweet_ids.txt', "a+") as file_object:
        file_object.seek(0)
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        file_object.write(text_to_append)

In [44]:
QUERY = "przyjaznybot"
MAX_RESULTS = 10

quote_tweets(QUERY, MAX_RESULTS)

[{'id': 1615408509134245888,
  'username': 'olalyricsbot',
  'text': '@przyjaznybot przyjazny draw lyrics'},
 {'id': 1615407576627257344,
  'username': 'przyjaznybot',
  'text': 'RT @olalyricsbot: testuje nowego bota'},
 {'id': 1615407573770899457,
  'username': 'przyjaznybot',
  'text': 'RT @olalyricsbot: @przyjaznybot draw lyrics'},
 {'id': 1615403081352478729,
  'username': 'olalyricsbot',
  'text': 'RT @olalyricsbot: @przyjaznybot draw lyrics'},
 {'id': 1615403077904760847,
  'username': 'olalyricsbot',
  'text': 'RT @olalyricsbot: testuje nowego bota'},
 {'id': 1615402988066963491,
  'username': 'olalyricsbot',
  'text': 'testuje nowego bota https://t.co/AOJ8RaV39u'},
 {'id': 1615402853689851916,
  'username': 'olalyricsbot',
  'text': '@przyjaznybot draw lyrics'}]

From list of dictionaries to list of specific variables

In [45]:
values_of_usernames = [dict['username'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_ids = [dict['id'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_text = [dict['text'] for dict in quote_tweets(QUERY, MAX_RESULTS)]

If someone share a tweet containing the phrase "przyjazny draw lyrics", bot create new tweet with the user's mention. <br>
If someone quotes a tweet, bot retweet it.

In [46]:
lines = []
lyrics = raw_lyrics(track_id)
for i in range(len(values_of_ids)):
    if check(str(values_of_ids[i])) == False:
        if values_of_usernames[i] != "przyjaznybot":
            if "przyjazny draw lyrics" in values_of_text[i] and check(str(values_of_ids[i])) == False:
                reply = "hello @" + values_of_usernames[i] + "\n" + "\n" + tweet_content(lyrics, track_id)
                api.like(values_of_ids[i])
                api.create_tweet(text=reply)
                append_new_id(str(values_of_ids[i]))
            else:
                api.like(values_of_ids[i])
                api.retweet(values_of_ids[i])
                append_new_id(str(values_of_ids[i]))

Searching for "GUGU x 2115" by 2115...
Done.
